In [1]:
include("./src/NeuralVerification.jl")

using .NeuralVerification, LazySets

In [2]:
nnet = read_nnet("./test/networks/small_nnet.nnet")

input_set = Hyperrectangle(low=[-1.], high=[1.])
output_set = Hyperrectangle(low=[-1.], high=[70.])

prob = Problem(nnet, input_set, output_set);

In [3]:
nnet.layers

3-element Array{Main.NeuralVerification.Layer,1}:
 Main.NeuralVerification.Layer{Main.NeuralVerification.ReLU,Float64}([1.0; 1.0], [1.5, 1.5], Main.NeuralVerification.ReLU())
 Main.NeuralVerification.Layer{Main.NeuralVerification.ReLU,Float64}([2.0 2.0; 2.0 2.0], [2.5, 2.5], Main.NeuralVerification.ReLU())
 Main.NeuralVerification.Layer{Main.NeuralVerification.Id,Float64}([3.0 3.0], [3.5], Main.NeuralVerification.Id())

In [4]:
reach = solve(DeepPoly(), prob)

4-element Array{Main.NeuralVerification.SymbolicInterval{HPolytope{Float64,Array{Float64,1}}},1}:
 Main.NeuralVerification.SymbolicInterval{HPolytope{Float64,Array{Float64,1}}}([1.0 0.0], [1.0 0.0], HPolytope{Float64,Array{Float64,1}}(HalfSpace{Float64,Array{Float64,1}}[HalfSpace{Float64,Array{Float64,1}}([1.0], 1.0), HalfSpace{Float64,Array{Float64,1}}([-1.0], 1.0)]))
 Main.NeuralVerification.SymbolicInterval{HPolytope{Float64,Array{Float64,1}}}([1.0 1.5; 1.0 1.5], [1.0 1.5; 1.0 1.5], HPolytope{Float64,Array{Float64,1}}(HalfSpace{Float64,Array{Float64,1}}[HalfSpace{Float64,Array{Float64,1}}([1.0], 1.0), HalfSpace{Float64,Array{Float64,1}}([-1.0], 1.0)]))
 Main.NeuralVerification.SymbolicInterval{HPolytope{Float64,Array{Float64,1}}}([4.0 8.5; 4.0 8.5], [4.0 8.5; 4.0 8.5], HPolytope{Float64,Array{Float64,1}}(HalfSpace{Float64,Array{Float64,1}}[HalfSpace{Float64,Array{Float64,1}}([1.0], 1.0), HalfSpace{Float64,Array{Float64,1}}([-1.0], 1.0)]))
 Main.NeuralVerification.SymbolicInterval{HP

In [5]:
import .NeuralVerification: init_deep_poly_symbolic_interval, bounds, upper, lower, n_nodes

In [6]:
for (index, r) in enumerate(reach)
    println("## Layer $(index - 1) ##")
    
    for d in 1:dim(r)
        symlo_bounds = bounds(lower(r), d)
        symhi_bounds = bounds(upper(r), d)
        println("symlo_$d ∈ $symlo_bounds, symhi_$d ∈ $symhi_bounds")
    end
end

## Layer 0 ##
symlo_1 ∈ (-1.0, 1.0), symhi_1 ∈ (-1.0, 1.0)
## Layer 1 ##
symlo_1 ∈ (0.5, 2.5), symhi_1 ∈ (0.5, 2.5)
symlo_2 ∈ (0.5, 2.5), symhi_2 ∈ (0.5, 2.5)
## Layer 2 ##
symlo_1 ∈ (4.5, 12.5), symhi_1 ∈ (4.5, 12.5)
symlo_2 ∈ (4.5, 12.5), symhi_2 ∈ (4.5, 12.5)
## Layer 3 ##
symlo_1 ∈ (30.5, 78.5), symhi_1 ∈ (30.5, 78.5)


# ACAS Test

Sanity check to see, if Julia implementation is correct.
Perform one symbolic forward pass through the first ACAS network and see, if the result matches the result of the python implementation.

In [11]:
# import NeuralPriorityOptimizer: get_acas_sets, optimize_linear, PriorityOptimizerParameters
include("../NeuralPriorityOptimizer.jl/src/NeuralPriorityOptimizer.jl")
using .NeuralPriorityOptimizer

Academic license - for non-commercial use only - expires 2022-07-17


In [8]:
acas = read_nnet("../NeuralPriorityOptimizer.jl\\networks\\CAS\\ACASXU_experimental_v2a_1_1.nnet")

input_set, output_set = get_acas_sets(1)

prob_acas = Problem(acas, input_set, output_set);

In [9]:
reach = solve(DeepPoly(), prob_acas);

r = reach[end]
for d in 1:5
    symlo_bounds = bounds(lower(r), d)
    symhi_bounds = bounds(upper(r), d)
    println("symlo_$d ∈ $symlo_bounds, symhi_$d ∈ $symhi_bounds")
end

symlo_1 ∈ (-7212.914208494338, -6928.804304024606), symhi_1 ∈ (27168.53293697694, 28068.438094652603)
symlo_2 ∈ (-12696.567257156463, -12168.824517303163), symhi_2 ∈ (31282.962650589365, 32318.010593996358)
symlo_3 ∈ (-8764.558778098499, -8431.682802031262), symhi_3 ∈ (32206.964320863874, 33240.39346864382)
symlo_4 ∈ (-17107.991080592954, -16484.088615046967), symhi_4 ∈ (29344.055613875775, 30318.165941502662)
symlo_5 ∈ (-11385.0064470153, -11013.51560234393), symhi_5 ∈ (29584.612365268185, 30536.836726927173)


In [10]:
params = PriorityOptimizerParameters(max_steps=10, stop_frequency=1, verbosity=1, stop_gap=0.001)
coeffs = zeros(5)
coeffs[1] = 1.;

In [38]:
function NeuralPriorityOptimizer.optimize_linear(network, input_set, coeffs, params; maximize=true, solver=DeepPoly())
    min_sign_flip = maximize ? 1.0 : -1.0
    approximate_optimize_cell = cell -> min_sign_flip * ρ(min_sign_flip .* coeffs, forward_network(solver, network, cell))
    achievable_value = cell -> (cell.center, compute_linear_objective(network, cell.center, coeffs))
    return general_priority_optimization(input_set, approximate_optimize_cell, achievable_value, params, maximize)
end

In [11]:
time = @elapsed x_star, lb, ub, steps = optimize_linear(acas, input_set, coeffs, params, solver=DeepPoly())

MethodError: MethodError: no method matching forward_linear(::DeepPoly, ::Main.NeuralVerification.Layer{Main.NeuralVerification.ReLU,Float64}, ::Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}})
Closest candidates are:
  forward_linear(!Matched::NeuralVerification.ExactReach, !Matched::NeuralVerification.Layer, ::Any) at C:\Users\phili\.juliapro\JuliaPro_v1.4.2-1\packages\NeuralVerification\w3flf\src\reachability\exactReach.jl:33
  forward_linear(!Matched::NeuralVerification.MaxSens, !Matched::NeuralVerification.Layer, ::Hyperrectangle) at C:\Users\phili\.juliapro\JuliaPro_v1.4.2-1\packages\NeuralVerification\w3flf\src\reachability\maxSens.jl:42
  forward_linear(!Matched::NeuralVerification.Ai2{HPolytope}, !Matched::NeuralVerification.Layer{NeuralVerification.ReLU,N} where N<:Number, ::AbstractPolytope) at C:\Users\phili\.juliapro\JuliaPro_v1.4.2-1\packages\NeuralVerification\w3flf\src\reachability\ai2.jl:60
  ...